<a href="https://colab.research.google.com/github/Jannathfatima/Bike-share/blob/main/Walmart_yf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=ad754e82237187a5a65d6004f5c1c7ed7dfdd10375743019eabf6dc26f46f628
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import yfinance as yf

In [3]:
stock = yf.Ticker('WMT')

In [4]:
data = stock.history(period='1d', start='2015-1-1', end= '2020-12-20')

In [5]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-01-02,74.876873,75.267448,74.251964,74.555740,4501800,0.0,0
2015-01-05,74.399523,74.920284,74.217257,74.338768,6979000,0.0,0
2015-01-06,74.625177,75.293485,74.460267,74.911591,8205100,0.0,0
2015-01-07,75.319526,76.968606,75.224052,76.899170,8498400,0.0,0
2015-01-08,77.428607,78.695794,77.307097,78.522209,12713600,0.0,0
...,...,...,...,...,...,...,...
2020-12-14,147.699997,147.720001,145.399994,145.649994,8362700,0.0,0
2020-12-15,145.520004,146.360001,144.860001,145.580002,10631500,0.0,0
2020-12-16,146.240005,146.889999,145.080002,145.429993,8550300,0.0,0


In [6]:
data1 = data.reset_index()['Close']

In [7]:
data1

0        74.555740
1        74.338768
2        74.911591
3        76.899170
4        78.522209
           ...    
1498    145.649994
1499    145.580002
1500    145.429993
1501    146.100006
1502    145.949997
Name: Close, Length: 1503, dtype: float64

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [9]:
scaler = MinMaxScaler(feature_range=(0,1))

In [10]:
data1 = scaler.fit_transform(np.array(data1).reshape(-1,1))

In [11]:
data1.shape

(1503, 1)

In [12]:
print(data1)

[[0.24084229]
 [0.23872179]
 [0.24432007]
 ...
 [0.93350787]
 [0.94005602]
 [0.93858995]]


In [13]:
#splitting the data into train and split
train_size = int(len(data1)*0.90)
test_size = len(data1)-train_size
train_data,test_data = data1[0:train_size,:],data1[train_size:len(data1),:1]

In [14]:
train_size, test_size

(1352, 151)

In [15]:
len(train_data),len(test_data)

(1352, 151)

In [16]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [17]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [18]:
print(X_train.shape), print(y_train.shape)

(1251, 100)
(1251,)


(None, None)

In [19]:
print(X_test.shape), print(ytest.shape)

(50, 100)
(50,)


(None, None)

In [20]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [21]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
regressor= Sequential()

In [22]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 50)           10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 71,051
Trainable params: 71,051
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 50)           10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 71,051
Trainable params: 71,051
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=200,batch_size=64,verbose=1)

Epoch 1/200
20/20 [==============================] - 14s 353ms/step - loss: 0.0569 - val_loss: 0.0067
Epoch 2/200
20/20 [==============================] - 5s 240ms/step - loss: 0.0053 - val_loss: 0.0018
Epoch 3/200
20/20 [==============================] - 5s 245ms/step - loss: 0.0014 - val_loss: 0.0132
Epoch 4/200
20/20 [==============================] - 5s 248ms/step - loss: 0.0011 - val_loss: 0.0053
Epoch 5/200
20/20 [==============================] - 5s 243ms/step - loss: 0.0010 - val_loss: 0.0076
Epoch 6/200
20/20 [==============================] - 5s 242ms/step - loss: 0.0011 - val_loss: 0.0072
Epoch 7/200
20/20 [==============================] - 5s 245ms/step - loss: 0.0010 - val_loss: 0.0062
Epoch 8/200
20/20 [==============================] - 5s 250ms/step - loss: 0.0010 - val_loss: 0.0042
Epoch 9/200
 6/20 [========>.....................] - ETA: 3s - loss: 0.0011

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
rmse=math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:

### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

In [ ]:
round((rmse))

In [ ]:

### Plotting 
# shift train predictions for plotting
from matplotlib import pyplot as plt
look_back=100
trainPredictPlot = numpy.empty_like(data1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(data1)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data1)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(data1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[51:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input


In [ ]:
## demonstrate prediction for next 30 days
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
len(data1)

In [ ]:
df3= data1.tolist()
df3.extend(lst_output)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(data1[1403:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df3=data1.tolist()
df3.extend(lst_output)
plt.plot(df3[1:])

In [ ]:
data.iloc[:,0:5]

In [ ]:
newdata=data.iloc[:,0:5]
newdata

In [ ]:
newdata['day'] = newdata.index.day
newdata['month'] = newdata.index.month
newdata['year'] = newdata.index.year
newdata

In [ ]:
#newdata.plot(figsize=(20,10), linewidth=5, fontsize=20)
#plt.xlabel('year', fontsize=20);


In [ ]:
newdata.corr()
